In [1]:
!pip install Flask pyngrok line-bot-sdk requests --quiet
!pip install google-genai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 8.8 MB/s eta 0:00:00


In [3]:
from google.colab import userdata

ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
line_channel_access_token = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = userdata.get('LINE_CHANNEL_SECRET')
gemini_api_key = userdata.get('GEMINI_API_KEY')
port = 5051

In [4]:
import os
from pyngrok import ngrok

In [5]:
ngrok.kill()

In [6]:
import requests

ngrok.set_auth_token(ngrok_authtoken)
tunnel = ngrok.connect(5051, name="linebot_tunnel")
webhook_url = tunnel.public_url

print(f"Ngrok URL: {webhook_url}")

# 自動更新 LINE Webhook URL
def update_line_webhook(webhook_url):
    """使用 LINE Messaging API 更新 Webhook URL"""
    url = "https://api.line.me/v2/bot/channel/webhook/endpoint"
    headers = {
        "Authorization": f"Bearer {line_channel_access_token}",
        "Content-Type": "application/json"
    }
    data = {
        "endpoint": webhook_url
    }

    response = requests.put(url, headers=headers, json=data)

    if response.status_code == 200:
        print(f"✅ LINE Webhook URL 已自動更新為：{webhook_url}")
        return True
    else:
        print(f"❌ 更新失敗：{response.status_code} - {response.text}")
        return False

# 執行更新
update_line_webhook(webhook_url)

Ngrok URL: https://noncalculative-semireactionary-debra.ngrok-free.dev
✅ LINE Webhook URL 已自動更新為：https://noncalculative-semireactionary-debra.ngrok-free.dev


True

In [7]:
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

# === 初始化 Google Gemini ===
client = genai.Client(api_key=gemini_api_key)

google_search_tool = Tool(
   google_search=GoogleSearch()
)

chat = client.chats.create(
    model="gemini-2.5-flash",
    config=GenerateContentConfig(
        system_instruction="你是一個中文的AI助手，請用繁體中文回答",
        tools=[google_search_tool],
        response_modalities=["TEXT"],
    )
)

In [8]:
def stateful_query(payload):
    response = chat.send_message(message=payload)
    return response.text

In [10]:
result = stateful_query("簡介RTSR(50字以內)")
print(result)

RTSR主要指光陽KYMCO高性能水冷街跑速克達RTS R 165，強調輕量、操控及賽道表現。亦涵蓋其統規賽事，驗證車輛實力並回饋研發。


In [11]:
result2 = stateful_query("RTSR是什麼？(50字以內)")
print(result2)

RTSR主要指光陽KYMCO的RTS R 165高性能水冷速克達，強調其輕量、操控及賽道表現。同時也指KYMCO舉辦的相關統規賽事。


In [12]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
)

app = Flask(__name__)

configuration = Configuration(access_token=line_channel_access_token)
handler = WebhookHandler(line_channel_secret)


@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    text = event.message.text
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        if text.startswith('AI '):
            prompt = text[3:]
            reply_text = stateful_query(prompt)
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=reply_text)]
                )
            )

        else:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=event.message.text),
                        TextMessage(text=event.message.text)]
                )
            )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5051
INFO:werkzeug:Press CTRL+C to quit


BODY:  {"destination":"U1adbf8b40af0e85791b2a893f8ea9ac6","events":[{"type":"message","message":{"type":"text","id":"594453871179333793","quoteToken":"3HMtVtWrSJP5rj-_PVNwsKrHF2PnrR2VaMNAtWoGQ_Gxv4F5lbxho26IHyDT8LyIzbdZsb0hHX8AsABT7T8Z_qVt407l0PYivw8YTU6jU_iI74P8vyTpf8EHmAmDVY6bnuFyy5sZ4PkF46JDIOcAkw","markAsReadToken":"cYu6-c-MinVm5HQTKyyt0fDWRDkdLMwMYDxek3RSUxNveqNY6nW3xKyKXAZTx_GmgAkjRg4D6syelIvWgLlAtYqvoaq5K9VJ9Y2PX5E-csMy0AY0Kkjvw3BmjoanslFQYXa3kOB1D_v6XWQvE39UFacFkY8gbUI99U17rRt0il-fiRtJGUVHnPZw_omwwdh-Sc0HuFYkrzRT3tRUOgU8BA","text":"AI 簡介RTSR(50字以內)"},"webhookEventId":"01KDS8TMFF8A1XNY8S83F29S3A","deliveryContext":{"isRedelivery":false},"timestamp":1767153422348,"source":{"type":"user","userId":"U4a7bcc8e7271a339e868a8b180a0945f"},"replyToken":"a67d1ef701af48fbb7151532c1e52aef","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [31/Dec/2025 03:57:04] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U1adbf8b40af0e85791b2a893f8ea9ac6","events":[{"type":"message","message":{"type":"text","id":"594453988133307041","quoteToken":"sqt5MqYZzicBqPcV0RAMJmiOMYC7uf90ByU3Lrq-fzqWdrlKyXXJVyca_Hay5vdpt7_shZgjKwcI1sn00fhAGOH5NSaL8x06zIqCXlc37KGhNHvSJ4uc_9-LCt7d-nQN7PpnH4Zhjc07dgzXbHogPg","markAsReadToken":"xTdQFTHN_5kizFO7kbIlAinmtwxA0iehSdEVczj_NBV_olyelvrvxJlqS0MkgCYypTNvxgUE8AWXHsBk9EXY1hS_Kxt3p84nQuPAXULRUfRT-S-dxp9is97l1E3Qq2soOD57Wl_eI1UeKRjd5Jh1DdLSjHsnZJ_3fdjdzVF8NBiE7pv6wo-y3Lwy-AnqP47EwwQN6Vha2KDXccJBWg02Uw","text":"AI RTSR是什麼?(50字以內)"},"webhookEventId":"01KDS8WR70F700M3CNMQBQZ0N3","deliveryContext":{"isRedelivery":false},"timestamp":1767153492111,"source":{"type":"user","userId":"U4a7bcc8e7271a339e868a8b180a0945f"},"replyToken":"dbe61f66ce8f4f3d9590552ec7c50636","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [31/Dec/2025 03:58:14] "POST / HTTP/1.1" 200 -
